In [ ]:
project/
│
├── app.py                # Main script to run the Streamlit app
├── data_loader.py        # Handles loading data
├── pages/
│   ├── about.py          # About the project page
│   ├── summary.py        # Data summary page
│   ├── unique_values.py  # Unique values page
│   ├── stats.py          # Basic statistics page
│   ├── demographics.py   # Demographics analysis page
│   ├── duration.py       # Process duration analysis page
│   ├── hypothesis.py     # Hypothesis testing page
│   ├── completion.py     # Completion time analysis page
│   ├── bounce_rate.py    # Bounce rate analysis page 
│   ├── error_rate.py
└── utils_
    ├── display.py        # Helper functions for displaying data

In [ ]:
# data_loader.py
import pandas as pd
import streamlit as st

@st.cache_data
def load_data():
    url = r"C:\Users\Cecilia\Downloads\ironhack\coursework\group_work\group_project_week5_6\second_project\data\clean\combined_cleaned_data1.csv"
    try:
        df = pd.read_csv(url)
        return df
    except FileNotFoundError:
        st.error("File not found. Please check the file path.")
        return None

In [ ]:
# app.py
import streamlit as st
from data_loader import load_data  # Assuming load_data is in the data_loader.py file
from pages import about, summary, unique_values, stats, demographics, hypothesistestcompletionrate, duration, completion, error_rate

def main():
    """
    Main function to run the Streamlit app for A/B Test Demo.
    
    Sets up the page configuration, loads data, and manages navigation
    through different pages of the app.
    """
    st.set_page_config(page_title="A/B Test Demo for Group 7")
    
    # Load the data here
    df = load_data()

    # If df is None, show an error and don't continue
    if df is None:
        st.error("Data could not be loaded.")
        return

    # Sort the data into control and test groups
    control_group = df[df['variation'] == 'Control']
    test_group = df[df['variation'] == 'Test']

    control_group_sorted = control_group.sort_values(by=['client_id', 'visit_id', 'process_step', 'date_time'])
    test_group_sorted = test_group.sort_values(by=['client_id', 'visit_id', 'process_step', 'date_time'])

    # Add age categorization (as it seems to be a necessary step in some of the analysis)
    bins = [0, 30, 40, 50, 100] 
    labels = ['Under 30', '30-39', '40-49', '50 and above']
    df['age_group'] = pd.cut(df['clnt_age'], bins=bins, labels=labels)
    control_group_sorted['age_group'] = pd.cut(control_group_sorted['clnt_age'], bins=bins, labels=labels)
    test_group_sorted['age_group'] = pd.cut(test_group_sorted['clnt_age'], bins=bins, labels=labels)

    st.sidebar.title("Navigation")
    page = st.sidebar.radio("Select a page:", [
        "About the Project", 
        "Data Summary", 
        "Unique Values", 
        "Basic Statistics", 
        "Demographics Analysis", 
        "Hypothesis Testing Completion Rate", 
        "Process Duration Analysis",  
        "Completion Time Analysis",
        "Error Rate Hypothesis Testing"  # Added Error Rate Hypothesis Testing
    ])

    # Handle page navigation and pass the df and sorted groups to the page
    if page == "About the Project":
        about.show_about_project()
    elif page == "Data Summary":
        summary.show_data_summary(df)
    elif page == "Unique Values":
        unique_values.show_unique_values_in_categorical_columns(df)
    elif page == "Basic Statistics":
        stats.show_basic_statistics(df)
    elif page == "Demographics Analysis":
        demographics.show_demographics(df, control_group_sorted, test_group_sorted)  # Pass sorted groups here
    elif page == "Hypothesis Testing Completion Rate":
        hypothesistestcompletionrate.show_page(df)
    elif page == "Process Duration Analysis":
        duration.show_process_duration(df)
    elif page == "Completion Time Analysis":
        completion.show_completion_time(df)
    elif page == "Error Rate Hypothesis Testing":
        error_rate.show_error_rate_analysis(df)  # Link to error_rate.py

if __name__ == "__main__":
    main()

In [ ]:
# pages/about.py
import streamlit as st

def show_about_project():
    st.title("About the Project")
    
    st.header("Project Overview")
    st.write(
        """
        An A/B test was set into motion from 3/15/2017 to 6/20/2017 by the Vanguard team.

        Control Group: Clients interacted with Vanguard’s traditional online process.
        Test Group: Clients experienced the new, spruced-up digital interface.

        * **Day 1 & 2 (Week 5)**  
          EDA & Data Cleaning  
          Client behavior analysis - explained below (trying to find relations and come up with hypotheses)

        * **Day 3 (Week 5)**  
          Performance Metrics  
          Success Indicators  
          Redesign Outcome

        * **Day 4 & 5 (Week 5)**  
          Hypothesis Testing  
          Completion Rate  
          Completion Rate with a Cost-Effectiveness Threshold  
          Other Hypothesis Examples  
          Experiment Evaluation  
          Design Effectiveness  
          Duration Assessment  
          Additional Data Needs

        * **Day 1 & 2 (Week 6)**  
          Tableau  
          Tableau Tasks

        * **Day 3 & 4 (Week 6)**
        """
    )

    st.header("Getting Started")
    st.write(
        """
        ## Metadata
        This comprehensive set of fields will guide your analysis, helping you unravel the intricacies of client behavior and preferences.

        - **client_id**: Every client’s unique ID.
        - **variation**: Indicates if a client was part of the experiment.
        - **visitor_id**: A unique ID for each client-device combination.
        - **visit_id**: A unique ID for each web visit/session.
        - **process_step**: Marks each step in the digital process.
        - **date_time**: Timestamp of each web activity.
        - **clnt_tenure_yr**: Represents how long the client has been with Vanguard, measured in years.
        - **clnt_tenure_mnth**: Further breaks down the client’s tenure with Vanguard in months.
        - **clnt_age**: Indicates the age of the client.
        - **gendr**: Specifies the client’s gender.
        - **num_accts**: Denotes the number of accounts the client holds with Vanguard.
        - **bal**: Gives the total balance spread across all accounts for a particular client.
        - **calls_6_mnth**: Records the number of times the client reached out over a call in the past six months.
        - **logons_6_mnth**: Reflects the frequency with which the client logged onto Vanguard’s platform over the last six months.

        ## Bonus: Additional Tasks (Optional)
        If you complete all of the tasks and have some extra time before the presentation, you can explore the following additional questions and tasks:

        - Client Behavior Analysis
        - Power and Effect Size
        - Streamlit  
          Add Streamlit to your project to achieve Customization and Real-time Analysis
        """
    )

In [ ]:
# pages/summary.py
import streamlit as st
import pandas as pd

def show_data_summary(df):
    st.subheader("CSV Data Overview")
    st.write(f"Number of rows: {df.shape[0]}")
    st.write(f"Number of columns: {df.shape[1]}")
    st.write("First 5 rows of the dataset:")
    st.dataframe(df.head())

# Assuming you're loading your data in the main part of the app or another script
if __name__ == "__main__":
    # Example: loading a CSV file
    # Change the path below to your actual file location
    df = pd.read_csv("path_to_your_data.csv")  # Load data from CSV

    # Now call the function and pass the DataFrame `df`
    show_data_summary(df)

In [ ]:
# pages/unique_values.py
import streamlit as st

def show_unique_values_in_categorical_columns(df):
    st.title("Unique Values in Categorical Columns")
    
    # Get all categorical columns
    categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()
    
    # Check if there are any categorical columns
    if not categorical_columns:
        st.warning("No categorical columns found in the file.")
        return

    st.subheader("Unique Values in Categorical Columns:")
    for column in categorical_columns:
        # Get unique values for each categorical column
        unique_values = df[column].unique()
        st.write(f"Column: {column}")
        st.write(f"Unique values: {unique_values}")

In [ ]:
# pages/stats.py
import streamlit as st

def show_basic_statistics(df):
    # Select only numeric columns
    numeric_df = df.select_dtypes(include=['number'])

    # Check if there are numeric columns
    if numeric_df.empty:
        st.warning("No numeric columns found in the file.")
        return
    
    # Display basic statistics for numeric columns
    st.subheader("Basic Statistics for Numeric Columns:")
    statistics = numeric_df.describe().T  # Transpose for better readability
    st.write(statistics)

In [ ]:
# pages/demographics.py
import pandas as pd
import plotly.express as px
import streamlit as st

# Function to perform demographic analysis
def analyze_demographics(df, control_group_sorted, test_group_sorted):
    """
    Function to perform demographic analysis and generate interactive plots using Plotly.
    """
    # Ensure 'clnt_age' is present and numeric
    if 'clnt_age' not in df.columns:
        st.error("The DataFrame does not contain the 'clnt_age' column.")
        return

    if not pd.api.types.is_numeric_dtype(df['clnt_age']):
        st.error("The 'clnt_age' column is not numeric.")
        return

    # Create the 'age_group' column based on 'clnt_age' ranges
    bins = [0, 18, 30, 40, 50, 60, 100]  # Define the age group ranges
    labels = ['0-18', '19-30', '31-40', '41-50', '51-60', '60+']  # Age group labels
    df['age_group'] = pd.cut(df['clnt_age'], bins=bins, labels=labels, right=False)

    # Debugging: Show first few rows of the dataframe to confirm 'age_group' column
    st.write("First few rows of the dataframe with 'age_group':")
    st.write(df[['clnt_age', 'age_group']].head())

    # Check if 'age_group' column exists now
    if 'age_group' not in df.columns:
        st.error("The 'age_group' column was not created.")
        return

    # Aggregating based on 'gender' and 'age_group'
    logs_calls_accounts = df.groupby(['gender', 'age_group']).agg({
        'num_accts': 'mean',
        'calls_6_mnth': 'mean',
        'logons_6_mnth': 'mean'
    }).reset_index().round(2)

    # Debugging: Show the aggregated result
    st.write("Aggregated data (grouped by 'gender' and 'age_group'):")
    st.write(logs_calls_accounts)

    # Plot for Average Number of Accounts
    fig1 = px.line(
        logs_calls_accounts, 
        x='age_group', 
        y='num_accts', 
        color='gender',
        title="Average Number of Accounts by Age Group and Gender",
        labels={'num_accts': 'Average Number of Accounts'},
        markers=True
    )
    st.plotly_chart(fig1)

    # Plot for Calls in the Last 6 Months
    fig2 = px.line(
        logs_calls_accounts, 
        x='age_group', 
        y='calls_6_mnth', 
        color='gender',
        title="Average Calls in Last 6 Months by Age Group and Gender",
        labels={'calls_6_mnth': 'Average Calls in Last 6 Months'},
        line_shape='linear',
        markers=True
    )
    st.plotly_chart(fig2)

    # Plot for Logons in the Last 6 Months
    fig3 = px.line(
        logs_calls_accounts, 
        x='age_group', 
        y='logons_6_mnth', 
        color='gender',
        title="Average Logons in Last 6 Months by Age Group and Gender",
        labels={'logons_6_mnth': 'Average Logons in Last 6 Months'},
        line_shape='linear',
        markers=True
    )
    st.plotly_chart(fig3)

    # *** Age Group by Test & Control ***
    st.write("### Test & Control Grouped by Age Group")
    # Filter based on unique client_id in control and test groups
    control_unique = control_group_sorted.drop_duplicates(subset='client_id')
    test_unique = test_group_sorted.drop_duplicates(subset='client_id')

    # Calculate age group distribution for each group
    control_age_group = control_unique["age_group"].value_counts()
    test_age_group = test_unique["age_group"].value_counts()

    # Combine data into one table
    age_groups_concat = pd.concat(
        [control_age_group, test_age_group], 
        axis=1, 
        keys=["Control Group Count", "Test Group Count"]
    )

    # Rename the columns for clarity
    age_groups_concat = age_groups_concat.sort_values(by="age_group", ascending=True)  # sort values
    age_groups_concat = age_groups_concat.reset_index()

    st.write(age_groups_concat)

    # *** Age Group x Gender ***
    st.write("### Age Group x Gender")
    control_age_group_gender = control_unique.groupby("age_group")["gender"].value_counts().unstack()
    test_age_group_gender = test_unique.groupby("age_group")["gender"].value_counts().unstack()

    # Reset the index to create a proper DataFrame structure
    control_age_group_gender = control_age_group_gender.reset_index()
    test_age_group_gender = test_age_group_gender.reset_index()

    st.write("Control Group - Age Group x Gender:")
    st.write(control_age_group_gender)

    st.write("Test Group - Age Group x Gender:")
    st.write(test_age_group_gender)

    # *** Age Group x Balances ***
    st.write("### Age Group x Balances")
    # Filter control and test group based on unique client_id
    control_age_group_balance = control_unique.groupby("age_group")["balance"].mean().round(2)
    test_age_group_balance = test_unique.groupby("age_group")["balance"].mean().round(2)

    # Convert the grouped Series to DataFrames
    control_age_group_balance_df = control_age_group_balance.reset_index()
    test_age_group_balance_df = test_age_group_balance.reset_index()

    # Rename the columns for clarity
    control_age_group_balance_df.rename(columns={"age_group": "Age Group", "balance": "Control Group Balance"}, inplace=True)
    test_age_group_balance_df.rename(columns={"age_group": "Age Group", "balance": "Test Group Balance"}, inplace=True)

    # Merge both control and test balance data into a single table
    balance_concat = pd.merge(control_age_group_balance_df, test_age_group_balance_df, on="Age Group")

    st.write(balance_concat)

# Function to display the demographics analysis in Streamlit
def show_demographics(df, control_group_sorted, test_group_sorted):
    """
    Show Demographics Analysis in the Streamlit app.
    This function is used to call the analysis and display the results.
    """
    st.title("Demographics Analysis")

    # Perform the demographic analysis (aggregation and plotting)
    analyze_demographics(df, control_group_sorted, test_group_sorted)

    # Additional notes or user guidance
    st.write("""
        This page provides demographic analysis, including average number of accounts, calls, 
        and logons, based on age groups and gender. The plots above allow you to explore how 
        these variables differ across age groups and between genders.
    """)

In [ ]:
#pages/duration.py
import pandas as pd
import streamlit as st

# Define the custom sorting order for the process steps
process_step_order = ['start', 'step_1', 'step_2', 'step_3', 'confirm']

# Convert 'process_step' to a categorical column with a custom sorting order
df_merged['process_step'] = pd.Categorical(df_merged['process_step'], categories=process_step_order, ordered=True)

# Filter groups based on test/control
control_group = df_merged[df_merged['variation'] == 'Control']
test_group = df_merged[df_merged['variation'] == 'Test']

# Sort the control group and test group by client_id, visit_id, process_step, and date_time
control_group_sorted = control_group.sort_values(by=['client_id', 'visit_id', 'process_step', 'date_time'])
test_group_sorted = test_group.sort_values(by=['client_id', 'visit_id', 'process_step', 'date_time'])

# Function to get the latest starts
def filter_latest_starts(group_df):
    # Filter the 'start' process step
    starts_only = group_df[group_df['process_step'] == 'start']
    
    # Get the latest 'start' for each 'visit_id' (group by visit_id, and get the row with max date_time)
    latest_starts = starts_only.loc[starts_only.groupby('visit_id')['date_time'].idxmax()]
    
    # Merge the latest starts back with the original dataframe to keep full process after the latest start
    return group_df.merge(latest_starts[['visit_id', 'date_time']], on=['visit_id', 'date_time'], how='inner')

# Apply to both groups (Control and Test)
filtered_control = filter_latest_starts(control_group_sorted)
filtered_test = filter_latest_starts(test_group_sorted)

# Display the complete tables for the filtered groups
st.title("Control Group Sorted and Filtered")
st.dataframe(filtered_control)

st.title("Test Group Sorted and Filtered")
st.dataframe(filtered_test)

# Check if it works for a specific client (e.g., client_id == 2304905)
client_total_entries = df_merged[df_merged["client_id"] == 2304905]
client_last_start_control = filtered_control[filtered_control['client_id'] == 2304905]
client_last_start_test = filtered_test[filtered_test['client_id'] == 2304905]

# Display the results for the specific client
st.title("Total Entries for Client 2304905")
st.dataframe(client_total_entries)

st.title("Last Start for Client 2304905 in Control Group")
st.dataframe(client_last_start_control)

st.title("Last Start for Client 2304905 in Test Group")
st.dataframe(client_last_start_test)

In [21]:
#pages/hypothesistestcompletionrate.py
import streamlit as st
import pandas as pd
import scipy.stats as stats

def show_page(df):
    """
    Function to display hypothesis testing results for completion rates
    between Control and Test groups.
    """

    # Check if the necessary columns exist
    if 'completion' not in df.columns:
        st.error("The 'completion' column is missing from the dataset.")
        return
    if 'variation' not in df.columns:
        st.error("The 'variation' column is missing from the dataset.")
        return

    # Filter the control and test groups from the dataframe
    control_group = df[df['variation'] == 'Control']
    test_group = df[df['variation'] == 'Test']

    # Calculate completion rates for both groups
    control_rate = control_group['completion'].mean()  # Assuming 'completion' column indicates completion
    test_rate = test_group['completion'].mean()

    # Display the completion rates
    st.subheader("Completion Rates")
    st.write(f"Control Group Completion Rate: {control_rate:.2%}")
    st.write(f"Test Group Completion Rate: {test_rate:.2%}")

    # Perform a hypothesis test to compare completion rates (e.g., z-test for proportions)
    # Null hypothesis: The completion rates are equal
    # Alternative hypothesis: The completion rates are different

    # Sample size for both groups
    n_control = len(control_group)
    n_test = len(test_group)

    # Perform z-test for proportions (for large samples, z-test is used to compare two proportions)
    control_successes = control_group['completion'].sum()
    test_successes = test_group['completion'].sum()

    # Perform z-test (z-test for two proportions)
    success_counts = [control_successes, test_successes]
    sample_sizes = [n_control, n_test]
    
    # Perform z-test for proportions
    z_stat, p_value = stats.proportions_ztest(success_counts, sample_sizes)

    # Display p-value and decision
    st.subheader("Hypothesis Test Results")
    st.write(f"P-value for comparison of completion rates: {p_value:.4f}")

    # Hypothesis test interpretation
    if p_value < 0.05:
        st.write("The difference in completion rates is statistically significant.")
    else:
        st.write("The difference in completion rates is not statistically significant.")

    # Optional: Plot completion rates for visual comparison
    st.subheader("Completion Rates Comparison")
    comparison_df = pd.DataFrame({
        'Group': ['Control', 'Test'],
        'Completion Rate': [control_rate, test_rate]
    })
    
    st.bar_chart(comparison_df.set_index('Group')['Completion Rate'])

    # Optional: Display a brief explanation of the hypothesis test
    st.subheader("Hypothesis Testing Explanation")
    st.write("""
    In this test, we are comparing the completion rates between the Control and Test groups.
    
    - **Null Hypothesis (H₀):** The completion rates for both groups are the same.
    - **Alternative Hypothesis (H₁):** The completion rates for both groups are different.
    
    A p-value less than 0.05 indicates that the null hypothesis can be rejected, meaning the difference in completion rates is statistically significant.
    """)

In [ ]:
#pages/completion.py
import streamlit as st
import pandas as pd  # Ensure pandas is imported

def show_completion_time(df):
    st.title("Completion Time Analysis")
    
    # Ensure the 'date_time' column is in datetime format
    if 'date_time' not in df.columns:
        st.error("Missing 'date_time' column in the dataset.")
        return
    
    # Coerce errors to NaT (Not a Time)
    df['date_time'] = pd.to_datetime(df['date_time'], errors='coerce')  

    # Drop rows where 'date_time' is NaT after coercion
    df = df.dropna(subset=['date_time'])

    # Sort the dataset to ensure chronological order by client_id, visit_id, and date_time
    df = df.sort_values(by=['client_id', 'visit_id', 'date_time'])
    
    # Split the data into control and test groups
    control_group = df[df['variation'] == 'Control']
    test_group = df[df['variation'] == 'Test']
    
    # Function to calculate completion time for each step considering multiple visits
    def calculate_completion_time_with_visits(group_df):
        group_df = group_df.sort_values(by=['client_id', 'visit_id', 'date_time'])
        group_df['next_step_time'] = group_df.groupby(['client_id', 'visit_id'])['date_time'].shift(-1)
        group_df = group_df.dropna(subset=['next_step_time'])
        group_df['completion_time'] = group_df['next_step_time'] - group_df['date_time']
        return group_df[['client_id', 'visit_id', 'process_step', 'date_time', 'next_step_time', 'completion_time']]

    # Function to filter out outliers using IQR
    def filter_outliers(group_df):
        # Convert completion_time to minutes for easier interpretation
        group_df['completion_time_minutes'] = group_df['completion_time'].dt.total_seconds() / 60
        
        # Calculate the IQR (Interquartile Range) for completion time
        Q1 = group_df['completion_time_minutes'].quantile(0.25)
        Q3 = group_df['completion_time_minutes'].quantile(0.75)
        IQR = Q3 - Q1

        # Define the upper and lower bounds for outliers
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Filter out the outliers based on IQR
        filtered_data = group_df[
            (group_df['completion_time_minutes'] >= lower_bound) &
            (group_df['completion_time_minutes'] <= upper_bound)
        ]
        return filtered_data

    # Apply the function to both the control and test groups
    control_group_completion_times = calculate_completion_time_with_visits(control_group)
    test_group_completion_times = calculate_completion_time_with_visits(test_group)
    
    # Filter out outliers from both groups
    control_group_filtered = filter_outliers(control_group_completion_times)
    test_group_filtered = filter_outliers(test_group_completion_times)
    
    # Calculate the average completion time in minutes for each process step, after removing outliers
    avg_completion_time_control = control_group_filtered.groupby('process_step')['completion_time_minutes'].mean().reset_index()
    avg_completion_time_test = test_group_filtered.groupby('process_step')['completion_time_minutes'].mean().reset_index()
    
    # Display the results for control and test group average completion times
    st.subheader("Average Completion Time for Control Group (Minutes) - After Outlier Removal")
    st.dataframe(avg_completion_time_control)

    st.subheader("Average Completion Time for Test Group (Minutes) - After Outlier Removal")
    st.dataframe(avg_completion_time_test)
    
    # Display comparison of control vs test group average completion times
    st.subheader("Comparison of Average Completion Time by Process Step")
    comparison_df = pd.merge(avg_completion_time_control, avg_completion_time_test, on='process_step', suffixes=('_control', '_test'))
    st.dataframe(comparison_df)

    # Optional: You could plot the results for better visualization if desired
    # st.bar_chart(comparison_df.set_index('process_step')[['completion_time_minutes_control', 'completion_time_minutes_test']])

    # Additional insights and interpretation
    st.subheader("Insights & Interpretation")
    st.write("""
    In this section, we can derive insights based on the completion time across different groups and process steps:
    
    - **Completion Time Analysis**: Are there significant differences in the time it takes for each process step between the Control and Test groups?
    - **Outlier Removal**: By removing outliers using the IQR method, we can focus on more representative data for process completion times.
    - **Improvement in Test Group**: Does the Test group show faster completion times compared to the Control group after considering the removal of outliers?
    
    Please review the data and interpret the results to drive business decisions and process improvements.
    """)

    # **Process Duration Analysis**: Calculate process duration for start and confirm steps
    st.subheader("Process Duration Analysis")

    # Filter to get the latest start for each client
    starts_only = df[df['process_step'] == 'start']
    latest_starts = starts_only.loc[starts_only.groupby('client_id')['date_time'].idxmax()]

    # Filter to get the last confirmation for each client
    confirmation_only = df[df['process_step'] == 'confirm']
    latest_confirms = confirmation_only.loc[confirmation_only.groupby('client_id')['date_time'].idxmax()]

    # Merge to have both latest start and confirm per client
    latest_start_confirms = pd.merge(latest_starts, latest_confirms, on='client_id', suffixes=('_start', '_confirm'))

    # Calculate process duration for those who completed the process
    latest_start_confirms['process_duration'] = latest_start_confirms['date_time_confirm'] - latest_start_confirms['date_time_start']

    # Convert timedelta to seconds for easier manipulation
    latest_start_confirms['process_duration_seconds'] = latest_start_confirms['process_duration'].dt.total_seconds()

    # Calculate the IQR (Interquartile Range) for process duration
    Q1_duration = latest_start_confirms['process_duration_seconds'].quantile(0.25)
    Q3_duration = latest_start_confirms['process_duration_seconds'].quantile(0.75)
    IQR_duration = Q3_duration - Q1_duration

    # Define the upper and lower bounds for outliers in process duration
    lower_bound_duration = Q1_duration - 1.5 * IQR_duration
    upper_bound_duration = Q3_duration + 1.5 * IQR_duration

    # Filter out the outliers based on IQR for process duration
    filtered_duration_data = latest_start_confirms[
        (latest_start_confirms['process_duration_seconds'] >= lower_bound_duration) &
        (latest_start_confirms['process_duration_seconds'] <= upper_bound_duration)
    ]

    # Convert process_duration back to Timedelta
    filtered_duration_data['process_duration'] = pd.to_timedelta(filtered_duration_data['process_duration_seconds'], unit='s')

    # Calculate the average process duration again after removing outliers
    st.subheader("Average Process Duration (Filtered) - After Outlier Removal")
    st.write(f"Average process duration: {filtered_duration_data['process_duration'].mean()}")
    st.write(f"Median process duration: {filtered_duration_data['process_duration'].median()}")
    
    # Optional: Display filtered duration data
    st.subheader("Filtered Process Duration Data (Outliers Removed)")
    st.dataframe(filtered_duration_data[['client_id', 'process_duration']])

In [ ]:
import streamlit as st
import pandas as pd
from scipy.stats import norm

# Function to calculate counts for z-test
def calculate_counts(group):
    steps = ['start', 'step_1', 'step_2', 'step_3', 'confirm']
    counts = []  # Store counts as tuples (N_started, N_dropped)
    
    for i in range(len(steps) - 1):
        current_step = steps[i]
        next_step = steps[i + 1]
        # Total users who started at this step
        started = group[group['process_step'] == current_step]['client_id'].nunique()
        # Total users who dropped off at this step
        reached_next = group[group['process_step'] == next_step]['client_id'].nunique()
        dropped = started - reached_next
        counts.append((started, dropped))
    
    return counts

# Function to perform two-proportion z-test
def two_proportion_z_test(n1, x1, n2, x2):
    # Calculate proportions
    p1 = x1 / n1 if n1 > 0 else 0
    p2 = x2 / n2 if n2 > 0 else 0
    
    # Pooled proportion
    p = (x1 + x2) / (n1 + n2)
    
    # Calculate z-statistic
    z = (p1 - p2) / ((p * (1 - p) * (1 / n1 + 1 / n2)) ** 0.5)
    
    # Calculate two-tailed p-value
    p_value = 2 * (1 - norm.cdf(abs(z)))
    
    return z, p_value

# Main function to display the bounce rate analysis page
def show_bounce_rate(df):
    st.title("Bounce Rate Analysis")

    # Ensure that the necessary columns exist in the dataframe
    if 'clnt_age' not in df.columns or 'process_step' not in df.columns or 'client_id' not in df.columns:
        st.error("Required columns ('clnt_age', 'process_step', 'client_id') are missing.")
        return
    
    # Create age groups based on 'clnt_age'
    bins = [0, 30, 40, 50, 100]  # You can adjust the age bins as needed
    labels = ['Under 30', '30-39', '40-49', '50 and above']
    df['age_group'] = pd.cut(df['clnt_age'], bins=bins, labels=labels)
    
    # Split the data into Control and Test groups
    control_group = df[df['variation'] == 'Control']
    test_group = df[df['variation'] == 'Test']
    
    # Sort Control and Test groups
    control_group_sorted = control_group.sort_values(by=['client_id', 'visit_id', 'process_step', 'date_time'])
    test_group_sorted = test_group.sort_values(by=['client_id', 'visit_id', 'process_step', 'date_time'])
    
    # Calculate the drop-off rates for both Control and Test groups
    control_dropoff_rate = calculate_dropoff_rate(control_group_sorted)
    test_dropoff_rate = calculate_dropoff_rate(test_group_sorted)

    # Display Bounce Rates for Control and Test Groups
    st.subheader("Bounce Rates for Control and Test Groups (Overall)")
    st.write("Control Group Bounce Rates (%):")
    for step, rate in control_dropoff_rate.items():
        st.write(f"{step}: {rate:.2f}%")
    
    st.write("\nTest Group Bounce Rates (%):")
    for step, rate in test_dropoff_rate.items():
        st.write(f"{step}: {rate:.2f}%")
    
    # Calculate and display drop-off rates by Age Group for Control and Test groups
    st.subheader("Bounce Rates by Age Group")

    control_dropoff_rate_by_age = calculate_dropoff_rate_by_age(control_group_sorted)
    test_dropoff_rate_by_age = calculate_dropoff_rate_by_age(test_group_sorted)

    # Display Control group drop-off rates by Age
    st.write("Control Group Bounce Rates by Age Group:")
    for age_group, rates in control_dropoff_rate_by_age.items():
        st.write(f"Age Group: {age_group}")
        for step, rate in rates.items():
            st.write(f"  {step}: {rate:.2f}%")
    
    # Display Test group drop-off rates by Age
    st.write("Test Group Bounce Rates by Age Group:")
    for age_group, rates in test_dropoff_rate_by_age.items():
        st.write(f"Age Group: {age_group}")
        for step, rate in rates.items():
            st.write(f"  {step}: {rate:.2f}%")

    # **Hypothesis Testing (Z-Test) Section**
    st.subheader("Hypothesis Test using Z-Test")
    
    # Null Hypothesis: H₀ = The bounce rates for the control and test groups are the same for a given step.
    # Alternative Hypothesis: H₁ = The bounce rates for the control and test groups are different for a given step.
    
    st.write("""
    **Null Hypothesis (H₀)**: The bounce rates for the control and test groups are the same for a given step.  
    **Alternative Hypothesis (H₁)**: The bounce rates for the control and test groups are different for a given step.
    """)

    # Calculate counts for Control and Test groups
    control_counts = calculate_counts(control_group_sorted)
    test_counts = calculate_counts(test_group_sorted)

    # Perform z-tests for each step and decide on hypothesis
    z_test_results = []

    steps = ['start', 'step_1', 'step_2', 'step_3']
    for i, step in enumerate(steps):
        n1, x1 = control_counts[i]  # Control group: (N_started, N_dropped)
        n2, x2 = test_counts[i]     # Test group: (N_started, N_dropped)
        
        # Perform z-test for proportions
        z_stat, p_value = two_proportion_z_test(n1, x1, n2, x2)
        
        # Decide whether to reject the null hypothesis
        reject_null = p_value < 0.05
        
        # Store results
        z_test_results.append({
            'Step': step,
            'Control Bounce Rate (%)': (x1 / n1) * 100 if n1 > 0 else 0,
            'Test Bounce Rate (%)': (x2 / n2) * 100 if n2 > 0 else 0,
            'Z-Statistic': z_stat,
            'P-Value': p_value,
            'Reject Null Hypothesis': reject_null
        })

    # Convert results to DataFrame for display
    z_test_results_df = pd.DataFrame(z_test_results)

    # Display Z-Test Results
    st.write(z_test_results_df)

    # Optional: Provide a brief explanation of the Z-Test
    st.write("""
    The Z-Test is used here to test if there is a significant difference between the bounce rates for the control and test groups at each process step.
    
    - A Z-Statistic closer to 0 indicates that the difference between the two groups is small.
    - A p-value below 0.05 suggests that we reject the null hypothesis and conclude that the bounce rates are significantly different.
    - If the p-value is greater than 0.05, we fail to reject the null hypothesis, meaning the bounce rates are similar.
    """)

In [ ]:
#pages/error_rate.py
import streamlit as st
import pandas as pd
from scipy.stats import binomtest, ttest_ind

# Function to calculate errors
def calculate_errors(group):
    group['step_index'] = group['process_step'].map({'start': 0, 'step_1': 1, 'step_2': 2, 'step_3': 3, 'confirm': 4})
    group['error'] = group['step_index'].diff().apply(lambda x: x < 0)  # Negative diff indicates a backward step
    return group

# Function to calculate error rates for both groups
def calculate_error_rates(control_group, test_group):
    # Calculate errors
    control_group = calculate_errors(control_group)
    test_group = calculate_errors(test_group)

    # Calculate Error Rates
    control_error_rate = control_group['error'].mean() * 100
    test_error_rate = test_group['error'].mean() * 100

    # Error rate difference
    error_rate_difference = control_error_rate - test_error_rate  # Difference between control and test error rates
    threshold = 5  # 5% threshold for the difference

    # Perform hypothesis testing: binomial test
    control_errors = control_group['error'].sum()
    control_total = len(control_group)
    test_errors = test_group['error'].sum()
    test_total = len(test_group)

    # Perform a one-tailed binomial test
    result = binomtest(test_errors, test_total, control_errors / control_total, alternative='less')

    # Perform independent t-test for error rates
    control_error_rate_values = control_group['error'].astype(int)
    test_error_rate_values = test_group['error'].astype(int)

    # Perform the independent t-test
    t_stat, t_p_value = ttest_ind(control_error_rate_values, test_error_rate_values, equal_var=False, alternative='two-sided')

    return {
        'control_error_rate': control_error_rate,
        'test_error_rate': test_error_rate,
        'error_rate_difference': error_rate_difference,
        'binomial_p_value': result.pvalue,
        't_statistic': t_stat,
        't_p_value': t_p_value,
    }

# Function to display the results
def show_error_rate_analysis(df):
    st.title("Error Rate Hypothesis Testing")

    # Split the data into control and test groups
    control_group = df[df['variation'] == 'Control']
    test_group = df[df['variation'] == 'Test']

    # Calculate error rates and perform hypothesis testing
    results = calculate_error_rates(control_group, test_group)

    # Display results
    st.subheader("Error Rates Comparison")
    st.write(f"Control Group Error Rate: {results['control_error_rate']:.2f}%")
    st.write(f"Test Group Error Rate: {results['test_error_rate']:.2f}%")
    st.write(f"Error Rate Difference: {results['error_rate_difference']:.2f}%")

    # Display hypothesis testing results
    st.subheader("Hypothesis Testing Results")

    # Binomial Test Results
    st.write(f"Binomial Test p-value: {results['binomial_p_value']:.4f}")
    if results['binomial_p_value'] < 0.05:
        st.write("The test group has significantly lower error rate than the control group (p-value < 0.05).")
    else:
        st.write("There is no significant difference in error rates between the test and control groups.")

    # T-test Results
    st.write(f"T-test Statistic: {results['t_statistic']:.4f}")
    st.write(f"T-test p-value: {results['t_p_value']:.4f}")
    if results['t_p_value'] < 0.05:
        st.write("There is a significant difference in error rates between the control and test groups.")
    else:
        st.write("There is no significant difference in error rates between the control and test groups.")

    # Conclusion based on practical significance (error rate difference)
    st.subheader("Practical Significance")
    if results['error_rate_difference'] >= 5:
        st.write("The test group has at least a 5% lower error rate than the control group, which is practically significant.")
    else:
        st.write("The error rate difference is less than 5%, which may not be practically significant for making decisions.")

    # Conclusion
    st.subheader("Conclusion")
    if results['binomial_p_value'] < 0.05 and results['error_rate_difference'] >= 5:
        st.write("The test group shows both statistical and practical significance. The improvement in error rates may justify action.")
    else:
        st.write("Although the test group shows statistically significant differences, the practical significance (error rate difference) may not justify making significant changes.")

In [ ]:
# utils/display.py
import streamlit as st

# Function to show an error message
def show_error(message: str):
    st.error(message)

# Function to display a dataframe
def display_dataframe(df, rows=5):
    if df is not None:
        st.dataframe(df.head(rows))
    else:
        show_error("Data is not available.")

# Function to display basic statistics of the dataframe
def show_basic_statistics(df):
    if df is not None:
        numeric_df = df.select_dtypes(include=['number'])
        if not numeric_df.empty:
            st.subheader("Basic Statistics")
            st.write(numeric_df.describe().T)
        else:
            show_error("No numeric columns found for statistics.")
    else:
        show_error("Data is not available for statistics.")

# Function to show unique values for categorical columns
def show_unique_values(df):
    if df is not None:
        categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()
        if categorical_columns:
            for column in categorical_columns:
                st.write(f"Column: {column}")
                st.write(f"Unique values: {df[column].unique()}")
        else:
            show_error("No categorical columns found.")
    else:
        show_error("Data is not available for unique value display.")